In [1]:
# baseado em https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/
# https://www.codigofluente.com.br/aula-06-scikit-learn-vizinhos-mais-proximos-knn/
# https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/classification/notebooks/Correcting%20for%20imbalanced%20datasets.ipynb#scrollTo=jq9376MGvTa7

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import cluster, neighbors, svm, metrics, preprocessing
import math
np.set_printoptions(suppress=True)

In [3]:
originalDF = pd.read_csv('penguins_size.csv', encoding = "ISO-8859-1")

In [4]:
from sklearn import preprocessing
 
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
encodedDF = originalDF.copy()
encodedDF['species']= label_encoder.fit_transform(encodedDF['species'])
encodedDF['island']= label_encoder.fit_transform(encodedDF['island'])
encodedDF['sex']= label_encoder.fit_transform(encodedDF['sex'])

encodedDF = encodedDF[~np.isnan(encodedDF).any(axis=1)]

In [5]:
from sklearn.preprocessing import StandardScaler

scaledArr = StandardScaler().fit_transform(encodedDF['island','culmen_length_mm','culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex'].values)
scaledDF = pd.DataFrame(data = scaledArr, columns = ['species', 'island','culmen_length_mm','culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']) 

KeyError: ('island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex')

In [ ]:
corr = scaledDF.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
y = scaledDF.loc[:, ['species']].values
X = scaledDF.loc[:, ['island','culmen_length_mm','culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']].values
X

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, scaledDF[['species']]], axis = 1)
label_encoder.inverse_transform(scaledDF['species'])

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Adelie', 'Chinstrap', 'Gentoo']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['species'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# n_neighbors=1: macro avg       0.92      0.92      0.92
# n_neighbors=2: macro avg       0.87      0.87      0.87
# n_neighbors=3: macro avg       0.89      0.88      0.87
# n_neighbors=4: macro avg       0.84      0.84      0.8
# n_neighbors=5: macro avg       0.78      0.80      0.78

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))